In [1]:
from natasha import Doc, Segmenter, NewsEmbedding, NewsMorphTagger

Код:

In [2]:
def program(text_path, query):

    with open(text_path, 'r', encoding='utf-8') as f:
        text = f.read()

    doc = Doc(text)
    
    segmenter = Segmenter()
    doc.segment(segmenter)
    
    tagger = NewsMorphTagger(NewsEmbedding())
    doc.tag_morph(tagger)

    
    pos_text = list(map(lambda token : token.pos, doc.tokens))
    poses = set(pos_text)
    pos_symbol_map = {}
    pointer = 0
    for pos in poses:
        pos_symbol_map[pos] = chr(ord('a') + pointer)
        pointer +=1

    def convert_to_symbols(poses):
        result = map(lambda pos : pos_symbol_map[pos], poses)
        return ''.join(result)

    symbol_text = convert_to_symbols(pos_text)
    symbol_query = convert_to_symbols(query.split())

    
    # путем подсчета z-функции ищет все вхождения sample в text за линейное время
    # мог бы использовать re.findall, но так быстрее
    def find_occurences(sample, text):
        s = sample + '$' + text

        n = len(s)
        z = [0] * n
        z[0] = 0
        l = r = 0

        # подсчет z-функции
        for i in range(1, n):
            k = max(min(z[i-l], r-i), 0)
            while i + k < n and s[i+k]==s[k]:
                k += 1
            z[i] = k
            if i + z[i] > r:
                l = i
                r = i + z[i]


        occurences = []
        for i in range(len(sample) + 1, n):
            if z[i] == len(sample):
                occurences.append(i - (len(sample) + 1))

        return occurences

    occurences = find_occurences(symbol_query, symbol_text)

    
    result = []
    for pointer in occurences:
        tokens = doc.tokens[pointer : pointer + len(symbol_query)]
        tokens = map(lambda token : token.text, tokens)
        result.append(' '.join(tokens))
    
    return result

Использование:

In [3]:
text_path = 'text.txt'

query = 'ADJ NOUN PUNCT'

In [4]:
result = program(text_path, query)
result

['английской девочки ,',
 '2020 года .',
 'оценён критиками ,',
 'визуальную составляющую .']